####  전이학습 
- 거대 이미지 데이터셋으로 사전학습된 모델을 활용
- ImageNet 데이터베이스 기반 모델들 사용
- 사용법
    * 모델의 특징추출부분(CNN)은 재사용
    * 모델의 학습 및 분류 부분(FC)은 커스텀화 사용

- [1] 모듈 로딩 <hr>

In [45]:
## 모듈로딩
import torch        
import torch.nn as nn
import torchvision.models as models         # 사전학습된 모델들 관련
from torchinfo import summary               # 모델 정보 관련 

- [2] 사전학습된 모델 로딩 및 설정 <hr>
    - 모델 사용 가중치 및 바이어스 설정
    - 층별 가중치 및 바이어스의 비활성화 설정

In [46]:
### 사전학습된 모델 로딩
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)


In [47]:
### 모델 구조 확인 
print(model)
summary(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Layer (type:depth-idx)                   Param #
VGG                                      --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       1,792
│    └─ReLU: 2-2                         --
│    └─Conv2d: 2-3                       36,928
│    └─ReLU: 2-4                         --
│    └─MaxPool2d: 2-5                    --
│    └─Conv2d: 2-6                       73,856
│    └─ReLU: 2-7                         --
│    └─Conv2d: 2-8                       147,584
│    └─ReLU: 2-9                         --
│    └─MaxPool2d: 2-10                   --
│    └─Conv2d: 2-11                      295,168
│    └─ReLU: 2-12                        --
│    └─Conv2d: 2-13                      590,080
│    └─ReLU: 2-14                        --
│    └─Conv2d: 2-15                      590,080
│    └─ReLU: 2-16                        --
│    └─MaxPool2d: 2-17                   --
│    └─Conv2d: 2-18                      1,180,160
│    └─ReLU: 2-19                

In [48]:
### 사전학습된 모델의 파라미터 비활성화 설정
for named, param in model.named_parameters():
    print(f'[{named}] - {param.shape}')
    # 역전파 시에 업데이트 되지 않도록 설정
    param.requires_grad=False

[features.0.weight] - torch.Size([64, 3, 3, 3])
[features.0.bias] - torch.Size([64])
[features.2.weight] - torch.Size([64, 64, 3, 3])
[features.2.bias] - torch.Size([64])
[features.5.weight] - torch.Size([128, 64, 3, 3])
[features.5.bias] - torch.Size([128])
[features.7.weight] - torch.Size([128, 128, 3, 3])
[features.7.bias] - torch.Size([128])
[features.10.weight] - torch.Size([256, 128, 3, 3])
[features.10.bias] - torch.Size([256])
[features.12.weight] - torch.Size([256, 256, 3, 3])
[features.12.bias] - torch.Size([256])
[features.14.weight] - torch.Size([256, 256, 3, 3])
[features.14.bias] - torch.Size([256])
[features.17.weight] - torch.Size([512, 256, 3, 3])
[features.17.bias] - torch.Size([512])
[features.19.weight] - torch.Size([512, 512, 3, 3])
[features.19.bias] - torch.Size([512])
[features.21.weight] - torch.Size([512, 512, 3, 3])
[features.21.bias] - torch.Size([512])
[features.24.weight] - torch.Size([512, 512, 3, 3])
[features.24.bias] - torch.Size([512])
[features.26.we

- [3] 사전학습모델의 분류 관련 층을 커스텀화 진행 <hr>

In [49]:
#model.classifier=nn.Linear(in_features=25088, out_features=5, bias=True)

In [50]:
for name, child in model.classifier.named_modules():
    print(name)


0
1
2
3
4
5
6


In [51]:


class My(nn.Module):
    def __init__(self):
        super().__init__()
        self.features=nn.Sequential(
            nn.Conv2d(),
            nn.Conv2d(),
            nn.ReLU(),
            nn.Conv2d(),
            nn.Conv2d(),
            nn.ReLU(),
            nn.Conv2d(),
            nn.ReLU()                       
        )
        
        self.classifier=nn.Sequential(
            nn.Linear(3, 5),
            nn.ReLU(),
            nn.Linear(3, 5),
            nn.ReLU()            
        )
        
    def forward(self, data):
        output=self.features(data)
        output=output.view(output.shape[0], -1)
        return self.classifier(output)

In [52]:
m1=My()
m1

TypeError: __init__() missing 3 required positional arguments: 'in_channels', 'out_channels', and 'kernel_size'